In [1]:
from urllib.request import urlopen
import pandas as pd

input_data = urlopen('https://featurelabs-static.s3.amazonaws.com/spam_text_messages_modified.csv')
data = pd.read_csv(input_data)
data.head()

,Category,Message
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [2]:
X=data.drop('Category',axis=1)
y=data['Category']

In [3]:
X.head(50)

,Message
0,Free entry in 2 a wkly comp to win FA Cup fina...
1,FreeMsg Hey there darling it's been 3 week's n...
2,WINNER!! As a valued network customer you have...
3,Had your mobile 11 months or more? U R entitle...
4,"SIX chances to win CASH! From 100 to 20,000 po..."
5,URGENT! You have won a 1 week FREE membership ...
6,"XXXMobileMovieClub: To use your credit, click ..."
7,England v Macedonia - dont miss the goals/team...
8,Thanks for your subscription to Ringtone UK yo...
9,07732584351 - Rodger Burns - MSG = We tried to...


In [4]:
y.value_counts(normalize=True)

ham     0.750084
spam    0.249916
Name: Category, dtype: float64

In [5]:
import evalml

In [8]:
X_train,X_test,y_train,y_test=evalml.preprocessing.split_data(X,y,problem_type='binary')

In [7]:
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [9]:
X_train.head()

,Message
562,"Haha I heard that, text me when you're around"
1253,I'm thinking that chennai forgot to come for a...
1816,Can you tell Shola to please go to college of ...
2054,K k pa Had your lunch aha.
511,staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323


In [10]:
from evalml import AutoMLSearch

In [12]:
automl=AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='binary',max_batches=1,optimize_thresholds=True)

In [13]:
automl.search()

In [14]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,2,Random Forest Classifier w/ Label Encoder + Na...,2,0.130812,0.036607,0.130812,98.485679,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,1,Logistic Regression Classifier w/ Label Encode...,1,0.204526,0.067735,0.204526,97.632337,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,0,Mode Baseline Binary Classification Pipeline,0,8.638305,0.025020,8.638305,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [15]:
automl.best_pipeline

pipeline = BinaryClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Natural Language Featurizer': ['Natural Language Featurizer', 'X', 'Label Encoder.y'], 'Imputer': ['Imputer', 'Natural Language Featurizer.x', 'Label Encoder.y'], 'Random Forest Classifier': ['Random Forest Classifier', 'Imputer.x', 'Label Encoder.y']}, parameters={'Label Encoder':{'positive_label': None}, 'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'boolean_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None, 'boolean_fill_value': None}, 'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1}}, random_seed=0)

In [16]:
best_pipeline = automl.best_pipeline

In [17]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


*************************************************************************************
* Random Forest Classifier w/ Label Encoder + Natural Language Featurizer + Imputer *
*************************************************************************************

Problem Type: binary
Model Family: Random Forest

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Natural Language Featurizer
3. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * boolean_impute_strategy : most_frequent
	 * categorical_fill_value : None
	 * numeric_fill_value : None
	 * boolean_fill_value : None
4. Random Forest Classifier
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for binary problems.
Total training time (including CV): 34.7 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary  Gini   AUC  Precision    F1  Balanced Accuracy Binary  Accuracy Binary # Training # Validation
0                      

In [18]:
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('binary'))
print(f'Accuracy Binary: {scores["Accuracy Binary"]}')

Accuracy Binary: 0.9765886287625418
